In [1]:
%run ../../shared_setup.ipynb

docker image cggh/biipy:v1.6.0


In [2]:
fasta

In [3]:
sorted(fasta.keys())

['Pf3D7_01_v3',
 'Pf3D7_02_v3',
 'Pf3D7_03_v3',
 'Pf3D7_04_v3',
 'Pf3D7_05_v3',
 'Pf3D7_06_v3',
 'Pf3D7_07_v3',
 'Pf3D7_08_v3',
 'Pf3D7_09_v3',
 'Pf3D7_10_v3',
 'Pf3D7_11_v3',
 'Pf3D7_12_v3',
 'Pf3D7_13_v3',
 'Pf3D7_14_v3']

In [4]:
fasta['Pf3D7_01_v3']

NpyFastaRecord(0..640851)

In [5]:
class TandemRepeatTable(etl.Table):
    
    def __init__(self, chrom, unit_length, min_tract_length):
        self.chrom = chrom
        self.unit_length = unit_length
        self.min_tract_length = min_tract_length
        
    def __iter__(self):
        chrom = self.chrom
        unit_length = self.unit_length
        min_tract_length = self.min_tract_length

        # header row
        yield ('chrom', 'start', 'stop', 'ru', 'nu', 'unit_length', 'tract_length')
        
        log('generating tandem repeats for chromosome', chrom, 'unit length', unit_length)

        # obtain sequence
        seq = fasta[chrom]
        
        # begin iteration
        i = 0
        while i < len(seq):
            
            # current repeat unit
            ru = seq[i:i+unit_length]
            
            # number of repeat units found
            nu = 1
            
            # next potential unit to compare
            next_unit = seq[i+(nu*unit_length):i+((nu+1)*unit_length)]
            
            # discover units
            while next_unit == ru:
                nu += 1
                next_unit = seq[i+(nu*unit_length):i+((nu+1)*unit_length)]
                
            # compute tract length
            tract_length = nu * unit_length
            
            # decide if discovered a tandem repeat
            if nu > 1 and tract_length > min_tract_length:
                yield (chrom, i+1, i+1+tract_length, ru, nu, unit_length, tract_length)
                i += nu * unit_length
            else:
                i += 1
        

In [6]:
tbl1 = TandemRepeatTable('Pf3D7_01_v3', 1, 6)
tbl1

2016-03-12 23:55:31.089804 :: generating tandem repeats for chromosome Pf3D7_01_v3 unit length 1
2016-03-12 23:55:31.239531 :: generating tandem repeats for chromosome Pf3D7_01_v3 unit length 1


0|chrom,1|start,2|stop,3|ru,4|nu,5|unit_length,6|tract_length
Pf3D7_01_v3,4179,4187,g,8,1,8
Pf3D7_01_v3,4337,4355,t,18,1,18
Pf3D7_01_v3,4411,4419,t,8,1,8
Pf3D7_01_v3,4633,4640,t,7,1,7
Pf3D7_01_v3,5457,5465,g,8,1,8


In [7]:
tbl2 = TandemRepeatTable('Pf3D7_01_v3', 2, 9)
tbl2

2016-03-12 23:55:32.625266 :: generating tandem repeats for chromosome Pf3D7_01_v3 unit length 2
2016-03-12 23:55:32.984365 :: generating tandem repeats for chromosome Pf3D7_01_v3 unit length 2


0|chrom,1|start,2|stop,3|ru,4|nu,5|unit_length,6|tract_length
Pf3D7_01_v3,4213,4225,tg,6,2,12
Pf3D7_01_v3,4337,4355,tt,9,2,18
Pf3D7_01_v3,27416,27446,aa,15,2,30
Pf3D7_01_v3,27499,27509,aa,5,2,10
Pf3D7_01_v3,27510,27524,aa,7,2,14


In [8]:
tbl_regions_1b

0|region_chrom,1|region_start,2|region_stop,3|region_type,4|region_size
Pf3D7_01_v3,1,27336,SubtelomericRepeat,27336
Pf3D7_01_v3,27337,92900,SubtelomericHypervariable,65564
Pf3D7_01_v3,92901,457931,Core,365031
Pf3D7_01_v3,457932,460311,Centromere,2380
Pf3D7_01_v3,460312,575900,Core,115589


In [16]:
#!rm -v /data/plasmodium/pfalciparum/pf-crosses/data/genome/sanger/version3/September_2012/Pf3D7_08_v3.tr.pickle

In [17]:
def tabulate_tr(chrom):
    fn = '/data/plasmodium/pfalciparum/pf-crosses/data/genome/sanger/version3/September_2012/%s.tr.pickle' % chrom
    if not os.path.exists(fn):
        tbl_tr = (
            etl
            .cat(*[
                TandemRepeatTable(chrom, 1, 6), 
                TandemRepeatTable(chrom, 2, 9), 
                TandemRepeatTable(chrom, 3, 11), 
                TandemRepeatTable(chrom, 4, 13), 
                TandemRepeatTable(chrom, 5, 14), 
                TandemRepeatTable(chrom, 6, 16), 
                TandemRepeatTable(chrom, 7, 18), 
                TandemRepeatTable(chrom, 8, 18), 
                TandemRepeatTable(chrom, 9, 18), 
                TandemRepeatTable(chrom, 10, 18), 
                TandemRepeatTable(chrom, 11, 18), 
                TandemRepeatTable(chrom, 12, 18), 
                TandemRepeatTable(chrom, 13, 18), 
                TandemRepeatTable(chrom, 14, 18), 
                TandemRepeatTable(chrom, 15, 18), 
                TandemRepeatTable(chrom, 16, 18), 
                TandemRepeatTable(chrom, 17, 18), 
                TandemRepeatTable(chrom, 18, 18), 
                TandemRepeatTable(chrom, 19, 18), 
                TandemRepeatTable(chrom, 20, 18), 
                TandemRepeatTable(chrom, 21, 18), 
                TandemRepeatTable(chrom, 22, 18), 
                TandemRepeatTable(chrom, 23, 18), 
                TandemRepeatTable(chrom, 24, 18), 
            ])
            # remove redundant sub-repeats
            .groupselectmin(key=('chrom', 'start'), value='unit_length')
            .intervalleftjoin(tbl_regions_1b, lkey='chrom', lstart='start', lstop='stop', include_stop=True,
                              rkey='region_chrom', rstart='region_start', rstop='region_stop')
            .cutout('region_chrom', 'region_start', 'region_stop', 'region_size')
        )
        tbl_tr.topickle(fn)
    tbl_tr = etl.frompickle(fn)
    return tbl_tr
    

In [18]:
tbl = tabulate_tr('Pf3D7_01_v3')
tbl

0|chrom,1|start,2|stop,3|ru,4|nu,5|unit_length,6|tract_length,7|region_type
Pf3D7_01_v3,13,34,cctaaac,3,7,21,SubtelomericRepeat
Pf3D7_01_v3,24,52,aaccctaaaccctg,2,14,28,SubtelomericRepeat
Pf3D7_01_v3,38,101,aaccctaaaccctgaacccta,3,21,63,SubtelomericRepeat
Pf3D7_01_v3,80,108,aaccctaaaccctg,2,14,28,SubtelomericRepeat
Pf3D7_01_v3,111,139,cctaaaccctgaac,2,14,28,SubtelomericRepeat


In [19]:
CHROMOSOMES

[b'Pf3D7_01_v3',
 b'Pf3D7_02_v3',
 b'Pf3D7_03_v3',
 b'Pf3D7_04_v3',
 b'Pf3D7_05_v3',
 b'Pf3D7_06_v3',
 b'Pf3D7_07_v3',
 b'Pf3D7_08_v3',
 b'Pf3D7_09_v3',
 b'Pf3D7_10_v3',
 b'Pf3D7_11_v3',
 b'Pf3D7_12_v3',
 b'Pf3D7_13_v3',
 b'Pf3D7_14_v3']

In [ ]:
tbl_tr_wg = etl.cat(*[tabulate_tr(str(chrom, 'ascii')) for chrom in CHROMOSOMES])

2016-03-12 23:56:23.906519 :: generating tandem repeats for chromosome Pf3D7_07_v3 unit length 1
2016-03-12 23:56:44.195917 :: generating tandem repeats for chromosome Pf3D7_07_v3 unit length 2
2016-03-12 23:57:01.922624 :: generating tandem repeats for chromosome Pf3D7_07_v3 unit length 3
2016-03-12 23:57:19.305757 :: generating tandem repeats for chromosome Pf3D7_07_v3 unit length 4
2016-03-12 23:57:36.235974 :: generating tandem repeats for chromosome Pf3D7_07_v3 unit length 5
2016-03-12 23:57:52.524319 :: generating tandem repeats for chromosome Pf3D7_07_v3 unit length 6
2016-03-12 23:58:08.520138 :: generating tandem repeats for chromosome Pf3D7_07_v3 unit length 7
2016-03-12 23:58:24.928209 :: generating tandem repeats for chromosome Pf3D7_07_v3 unit length 8
2016-03-12 23:58:41.083142 :: generating tandem repeats for chromosome Pf3D7_07_v3 unit length 9


In [ ]:
tbl_tr_wg

## Fraction of core genome in STRs

In [ ]:
# TODO recode below

In [18]:
tbl_exons

0|feature_chrom,1|feature_type,2|feature_start,3|feature_stop,4|feature_length,5|feature_strand,6|feature_id,7|feature_parent_id,8|feature_name,9|feature_previous_id,10|feature_region_type,11|feature_region_size
Pf3D7_01_v3,CDS,29510,34762,5252,+,PF3D7_0100100.1:exon:1,PF3D7_0100100.1,None,None,SubtelomericHypervariable,65564
Pf3D7_01_v3,CDS,35888,37126,1238,+,PF3D7_0100100.1:exon:2,PF3D7_0100100.1,None,None,SubtelomericHypervariable,65564
Pf3D7_01_v3,CDS,38982,39923,941,-,PF3D7_0100200.1:exon:1,PF3D7_0100200.1,None,None,SubtelomericHypervariable,65564
Pf3D7_01_v3,CDS,40154,40207,53,-,PF3D7_0100200.1:exon:2,PF3D7_0100200.1,None,None,SubtelomericHypervariable,65564
Pf3D7_01_v3,CDS,42367,43617,1250,-,PF3D7_0100300.1:exon:1,PF3D7_0100300.1,None,None,SubtelomericHypervariable,65564


In [32]:
tbl_tr_core = (
    tbl_tr_wg
    .eq('region_type', 'Core')
    .rename('pos', 'start')
    .addfield('stop', lambda row: row.start + row.tract_length, index=2)
    .intervalleftjoin(tbl_exons.cutout('feature_type', 'feature_region_type', 'feature_region_size'), 
                      lkey='chrom', lstart='start', lstop='stop',
                      rkey='feature_chrom', rstart='feature_start', rstop='feature_stop', 
                      include_stop=True)
    .cutout('feature_chrom')
    .cache()
)
tbl_tr_core

0|chrom,1|start,2|stop,3|ru,4|nu,5|unit_length,6|tract_length,7|context,8|region_type,9|feature_start,10|feature_stop,11|feature_length,12|feature_strand,13|feature_id,14|feature_parent_id,15|feature_name,16|feature_previous_id
Pf3D7_01_v3,93098,93105,a,7,1,7,caaaaaaat,Core,None,None,None,None,None,None,None,None
Pf3D7_01_v3,93902,93940,at,19,2,38,taatatatatatatatatatatatatatatatatatatataa,Core,None,None,None,None,None,None,None,None
Pf3D7_01_v3,94016,94023,a,7,1,7,taaaaaaat,Core,None,None,None,None,None,None,None,None
Pf3D7_01_v3,94258,94282,at,12,2,24,aaatatatatatatatatatatatataa,Core,None,None,None,None,None,None,None,None
Pf3D7_01_v3,94433,94445,taa,4,3,12,ttttaataataataatat,Core,None,None,None,None,None,None,None,None


In [34]:
tbl_tr_core.nrows()

204289

In [35]:
tbl_tr_core.values('tract_length').sum()

2966687

In [36]:
tbl_tr_core.notnone('feature_id')

0|chrom,1|start,2|stop,3|ru,4|nu,5|unit_length,6|tract_length,7|context,8|region_type,9|feature_start,10|feature_stop,11|feature_length,12|feature_strand,13|feature_id,14|feature_parent_id,15|feature_name,16|feature_previous_id
Pf3D7_01_v3,98886,98893,a,7,1,7,gaaaaaaat,Core,98819,99013,194,+,PF3D7_0102200.1:exon:1,PF3D7_0102200.1,None,None
Pf3D7_01_v3,99591,99599,a,8,1,8,taaaaaaaat,Core,99220,102282,3062,+,PF3D7_0102200.1:exon:2,PF3D7_0102200.1,None,None
Pf3D7_01_v3,100030,100037,a,7,1,7,gaaaaaaag,Core,99220,102282,3062,+,PF3D7_0102200.1:exon:2,PF3D7_0102200.1,None,None
Pf3D7_01_v3,100060,100068,g,8,1,8,tgggggggga,Core,99220,102282,3062,+,PF3D7_0102200.1:exon:2,PF3D7_0102200.1,None,None
Pf3D7_01_v3,100759,100766,a,7,1,7,taaaaaaag,Core,99220,102282,3062,+,PF3D7_0102200.1:exon:2,PF3D7_0102200.1,None,None


In [64]:
n_core_coding_tr = tbl_tr_core.notnone('feature_id').nrows()
n_core_coding_tr

53648

In [65]:
bp_core_coding_tr = tbl_tr_core.notnone('feature_id').values('tract_length').sum()
bp_core_coding_tr

580324

In [39]:
tbl_tr_core.none('feature_id')

0|chrom,1|start,2|stop,3|ru,4|nu,5|unit_length,6|tract_length,7|context,8|region_type,9|feature_start,10|feature_stop,11|feature_length,12|feature_strand,13|feature_id,14|feature_parent_id,15|feature_name,16|feature_previous_id
Pf3D7_01_v3,93098,93105,a,7,1,7,caaaaaaat,Core,None,None,None,None,None,None,None,None
Pf3D7_01_v3,93902,93940,at,19,2,38,taatatatatatatatatatatatatatatatatatatataa,Core,None,None,None,None,None,None,None,None
Pf3D7_01_v3,94016,94023,a,7,1,7,taaaaaaat,Core,None,None,None,None,None,None,None,None
Pf3D7_01_v3,94258,94282,at,12,2,24,aaatatatatatatatatatatatataa,Core,None,None,None,None,None,None,None,None
Pf3D7_01_v3,94433,94445,taa,4,3,12,ttttaataataataatat,Core,None,None,None,None,None,None,None,None


In [66]:
n_core_noncoding_tr = tbl_tr_core.none('feature_id').nrows()
n_core_noncoding_tr

150641

In [67]:
bp_core_noncoding_tr = tbl_tr_core.none('feature_id').values('tract_length').sum()
bp_core_noncoding_tr

2386363

In [73]:
bp_core_coding = tbl_exons.eq('feature_region_type', 'Core').values('feature_length').sum()
bp_core_coding

11674189

In [69]:
tbl_introns

0|feature_chrom,1|feature_type,2|feature_start,3|feature_stop,4|feature_length,5|feature_strand,6|feature_id,7|feature_parent_id,8|feature_name,9|feature_previous_id,10|feature_region_size,11|feature_region_type,12|region_size
Pf3D7_01_v3,intron,34762,35888,1126,+,PF3D7_0100100.1:exon:1_PF3D7_0100100.1:exon:2,PF3D7_0100100.1,None,None,65564,SubtelomericHypervariable,65564
Pf3D7_01_v3,intron,39923,40154,231,-,PF3D7_0100200.1:exon:1_PF3D7_0100200.1:exon:2,PF3D7_0100200.1,None,None,65564,SubtelomericHypervariable,65564
Pf3D7_01_v3,intron,43617,43775,158,-,PF3D7_0100300.1:exon:1_PF3D7_0100300.1:exon:2,PF3D7_0100300.1,None,None,65564,SubtelomericHypervariable,65564
Pf3D7_01_v3,intron,50416,50572,156,+,PF3D7_0100400.1:exon:1_PF3D7_0100400.1:exon:2,PF3D7_0100400.1,None,None,65564,SubtelomericHypervariable,65564
Pf3D7_01_v3,intron,54788,54938,150,-,PF3D7_0100600.1:exon:1_PF3D7_0100600.1:exon:2,PF3D7_0100600.1,None,None,65564,SubtelomericHypervariable,65564


In [70]:
tbl_intergenic

0|feature_chrom,1|feature_type,2|feature_start,3|feature_stop,4|feature_length,5|feature_id,6|feature_region_size,7|feature_region_type,8|region_size
Pf3D7_01_v3,intergenic_0,37126,38982,1856,PF3D7_0100100_PF3D7_0100200,65564,SubtelomericHypervariable,65564
Pf3D7_01_v3,intergenic_1,40207,42367,2160,PF3D7_0100200_PF3D7_0100300,65564,SubtelomericHypervariable,65564
Pf3D7_01_v3,intergenic_2,46507,50363,3856,PF3D7_0100300_PF3D7_0100400,65564,SubtelomericHypervariable,65564
Pf3D7_01_v3,intergenic_0,51636,53169,1533,PF3D7_0100400_PF3D7_0100500,65564,SubtelomericHypervariable,65564
Pf3D7_01_v3,intergenic_1,53280,53778,498,PF3D7_0100500_PF3D7_0100600,65564,SubtelomericHypervariable,65564


In [71]:
tbl_core_noncoding = (
    etl
    .cat(tbl_introns, tbl_intergenic)
    .sort(key=('feature_chrom', 'feature_start'))
    .eq('feature_region_type', 'Core')
    .cutout('feature_region_size', 'feature_region_type', 'region_size')
    .cache()
)
tbl_core_noncoding

0|feature_chrom,1|feature_type,2|feature_start,3|feature_stop,4|feature_length,5|feature_strand,6|feature_id,7|feature_parent_id,8|feature_name,9|feature_previous_id
Pf3D7_01_v3,intergenic_1,91420,93113,1693,None,PF3D7_0101900_PF3D7_0102000,None,None,None
Pf3D7_01_v3,intergenic_2,93875,95050,1175,None,PF3D7_0102000_PF3D7_0102100,None,None,None
Pf3D7_01_v3,intergenic_1,95899,98819,2920,None,PF3D7_0102100_PF3D7_0102200,None,None,None
Pf3D7_01_v3,intron,99013,99220,207,+,PF3D7_0102200.1:exon:1_PF3D7_0102200.1:exon:2,PF3D7_0102200.1,None,None
Pf3D7_01_v3,intergenic_1,102282,104704,2422,None,PF3D7_0102200_PF3D7_0102300,None,None,None


In [74]:
bp_core_noncoding = tbl_core_noncoding.values('feature_length').sum()
bp_core_noncoding

9225374

In [76]:
pc_core_coding_tr = bp_core_coding_tr * 100 / bp_core_coding
pc_core_coding_tr

4.971000555156337

In [78]:
pc_core_noncoding_tr = bp_core_noncoding_tr * 100 / bp_core_noncoding
pc_core_noncoding_tr

25.867384888677684

In [79]:
bp_core_coding / n_coding_tr

217.60716149716671

In [80]:
bp_core_noncoding / n_noncoding_tr

61.24079101970911

In [82]:
bp_core_coding_tr / n_core_coding_tr

10.817253206084104

In [83]:
bp_core_noncoding_tr / n_core_noncoding_tr

15.841391121938914

## Are longer STRs more polymorphic?

In [ ]:
tbl_tr_wg = etl.cat(*[tabulate_tr(str(chrom, 'ascii')) for chrom in CHROMOSOMES)
tbl_tr_wg

In [ ]:
tbl_tr_core = tbl_tr_wg.eq('region_type', 'Core').cutout('region_type')
tbl_tr_core

In [ ]:
def tabulate_indels(cross):
    
    # VCF file name
    vcf_fn = COMBINED_VCF_FN_TEMPLATE.format(cross=cross)
    
    # build the table
    tbl = (etl
        .fromvcf(vcf_fn, samples=None)
        # only keep PASS variants
        .false('FILTER')
        # exclude fields not needed in this analysis
        .cutout('ID', 'QUAL', 'FILTER', 'INFO')
        # only keep INDELs
        .select(lambda row: len(row.REF) > 1 or any(len(a) > 1 for a in row.ALT))
        .addfield('cross', cross)        
    )
    
    return tbl    

In [ ]:
tbl_indels = (
    etl
    .cat(*[tabulate_indels(cross) for cross in CROSSES])
    .sort(key=('CHROM', 'POS'))
)
tbl_indels

In [ ]:
tbl_tr_indel = (
    tbl_tr_core
    .intervalleftjoin(tbl_indels.addfield('stop', lambda row: row.POS+1), 
                      lkey='chrom', lstart='start', lstop='stop',
                      rkey='CHROM', rstart='POS', rstop='stop', include_stop=True)
)
tbl_tr_indel.display(50)

In [ ]:
tbl_analysis = (
    tbl_tr_indel
    .distinct(key=(0, 1, 2), presorted=True)
    .addfield('polymorphic', lambda row: row.ALT is not None)
    .cutout('CHROM', 'POS', 'REF', 'ALT', 'cross', 12)
)
tbl_analysis.display(20)

In [ ]:
df = tbl_analysis.todataframe()
df.head()

In [ ]:
unit_length = 2
x = df[df.unit_length == unit_length].tract_length
plt.hist(x, bins=np.arange(unit_length, x.max(), unit_length))
y = df[(df.unit_length == unit_length) & df.polymorphic].tract_length
plt.hist(y, bins=np.arange(unit_length, x.max(), unit_length));

In [ ]:
fig, ax = plt.subplots()
for unit_length in 1, 2, 3, 4, 5, 6:
    x1 = df[df.unit_length == unit_length].tract_length
    x2 = df[(df.unit_length == unit_length) & df.polymorphic].tract_length
    b = np.arange(unit_length, x.max(), unit_length)
    h1, _ = np.histogram(x1, bins=b)
    h2, _ = np.histogram(x2, bins=b)
    f = h2 / h1
    ax.plot(b[:-1], f, label=unit_length, lw=2)
ax.set_xlim(0, 40)
ax.set_xlabel('STR tract length')
ax.set_ylabel('fraction polymorphic')
ax.legend(bbox_to_anchor=(1, 1), loc='upper left', title='STR unit length');